In [1]:

import cv2
import numpy as np

# 帧提取函数
def frame_generator(video_paths):
    captures = [cv2.VideoCapture(video_path) for video_path in video_paths]
    while True:
        frames = []
        ret_vals = []
        for capture in captures:
            ret, frame = capture.read()
            frames.append(frame)
            ret_vals.append(ret)
        if all(ret_vals):
            yield frames
        else:
            break
    for capture in captures:
        capture.release()

# 特征检测和匹配函数
def feature_detection_and_matching(frame1, frame2):
    orb = cv2.ORB_create()
    kp1, des1 = orb.detectAndCompute(frame1, None)
    kp2, des2 = orb.detectAndCompute(frame2, None)
    
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des1, des2)
    matches = sorted(matches, key=lambda x: x.distance)
    
    pts1 = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    pts2 = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
    
    return pts1, pts2

# 计算单应性矩阵函数
def calculate_homography(pts1, pts2):
    H, _ = cv2.findHomography(pts1, pts2, cv2.RANSAC)
    return H

# 图像拼接函数
def image_stitching(frames, H):
    h1, w1 = frames[0].shape[:2]
    h2, w2 = frames[1].shape[:2]
    
    result = cv2.warpPerspective(frames[1], H, (w1 + w2, h1))
    result[0:h1, 0:w1] = frames[0]
    
    return result

# 主函数
def main(video_paths):
    generator = frame_generator(video_paths)
    prev_H = np.eye(3)
    for frames in generator:
        pts1, pts2 = feature_detection_and_matching(frames[0], frames[1])
        H = calculate_homography(pts1, pts2)
        # 使用动量更新
        H = 0.8 * prev_H + 0.2 * H
        result = image_stitching(frames, H)
        prev_H = H
        cv2.imwrite('Stitched Frame', result)
        # if cv2.waitKey(1) & 0xFF == ord('q'):
            # break
    # cv2.destroyAllWindows()

if __name__ == "__main__":
    video_paths = ["../data/testolabc1.avi", "../data/testolabc2.avi"]
    main(video_paths)

error: OpenCV(4.10.0) /io/opencv/modules/imgcodecs/src/loadsave.cpp:708: error: (-2:Unspecified error) could not find a writer for the specified extension in function 'imwrite_'
